In [ ]:
import awe.utils
awe.utils.init_notebook()

In [ ]:
import awe.data.set.label_validation
import awe.data.set.swde
import awe.data.set.apify
awe.utils.reload('awe.data')

In [ ]:
ds = awe.data.set.apify.Dataset()

In [ ]:
df = ds.verticals[0].websites[0].df

In [ ]:
ds = awe.data.set.swde.Dataset(suffix='-exact')

In [ ]:
page = ds.verticals[0].websites[0].pages[0]
page

In [ ]:
page.url, page.html_path

In [ ]:
page_labels = page.get_labels()
{
    k: page_labels.get_label_values(k)
    for k in page_labels.label_keys
}

In [ ]:
{
    k: [
        n.text()
        for n in page_labels.get_labeled_nodes(k)
    ]
    for k in page_labels.label_keys
}

In [ ]:
pages = ds.get_all_pages(zip_verticals=True, zip_websites=True)
awe.data.set.label_validation.validate(pages)